In [1]:
!pip install unsloth

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-26 09:24:52.906387: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 09:24:52.915431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740551092.924422   10620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740551092.927071   10620 cuda_blas.cc

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-1.7B-Instruct-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.584 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


In [7]:

from datasets import load_dataset
# 2. Tapaco Veri Setini Yükleyin
dataset = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/")  # Tapaco veri setini yükle
#dataset2 = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})
#dataset3 = load_dataset("kayrab/patient-doctor-qa-tr-167732",  cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question_content": "Input", "question_answer": "Output"})

In [8]:
#dataset3 = concatenate_datasets([dataset3["train"], dataset3["test"]])
#dataset = concatenate_datasets([dataset, dataset2, dataset3]).remove_columns(['id', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx', 'doctor_title', 'doctor_speciality'])

In [9]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["Input"] is not None and example["Output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

In [10]:
dataset

Dataset({
    features: ['Input', 'Output'],
    num_rows: 2575505
})

In [11]:
def questions2gptFormat(rows):

    conversations = []
    
    def format_row(row):
        #print(row)
        return conversations.append( [{"from": "user", "value": row[0]}, {"from": "assistant", "value": row[1]}] )
                
           
    for i in zip(rows["Input"], rows["Output"]):
      
        format_row(i)

    return {"conversations":conversations}


In [12]:
dataset = dataset.map(questions2gptFormat, batched = True, batch_size=10000).remove_columns(["Input","Output"])

In [13]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 2575505
})

In [14]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [15]:
""" from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,) """

' from unsloth.chat_templates import standardize_sharegpt\ndataset = standardize_sharegpt(dataset)\ndataset = dataset.map(formatting_prompts_func, batched = True,) '

In [16]:
dataset[5]["conversations"]

[{'from': 'user',
  'value': 'Şu soruyu cevaplayın: Yeni 10 dolarlık banknot ne zaman çıkacak?'},
 {'from': 'assistant',
  'value': 'Merhaba küçük dostum! Yeni 10 dolarlık banknotun çıkış tarihi henüz açıklanmadı. ABD Hazine Bakanlığı adı verilen parayı kazanan kişilerin, hazır olduğunda bize haber vermesini beklemek zorunda kalacağız. Şimdilik mevcut 10 dolarlık banknotları kullanmaya devam edebiliriz.'}]

In [17]:
print(dataset[5]["text"])

<|im_start|>system
Şu soruyu cevaplayın: Yeni 10 dolarlık banknot ne zaman çıkacak?<|im_end|>
<|im_start|>system
Merhaba küçük dostum! Yeni 10 dolarlık banknotun çıkış tarihi henüz açıklanmadı. ABD Hazine Bakanlığı adı verilen parayı kazanan kişilerin, hazır olduğunda bize haber vermesini beklemek zorunda kalacağız. Şimdilik mevcut 10 dolarlık banknotları kullanmaya devam edebiliriz.<|im_end|>



In [18]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

In [19]:
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()


In [20]:
""" import wandb
from transformers import TrainerCallback
import torch
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

class WandBQuestionCallback(TrainerCallback):
    def __init__(self, tokenizer, model, questions, log_interval=500,**kwargs):
        self.tokenizer = get_chat_template(
            tokenizer,
            chat_template="chatml",
            mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
            map_eos_token=True,
        )
        
        self.model = model

        self.device = "cuda"
        self.questions = questions  # List of question strings
        self.log_interval = log_interval
        
    def on_step_end(self, args, state, control, **kwargs):
        # if state.global_step % self.log_interval == 0:
        #     wandb.log({"step": state.global_step})
        #     self.log_model_responses()
        if True:
            wandb.log({"step": state.global_step})
            self.log_model_responses()

    def log_model_responses(self,**kwargs):

        

        responses = {}
        for question in self.questions:
            messages = [
                {"from": "human", "value": question},
            ]
            inputs = self.tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(self.device)
            
            outputs = self.model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.2, top_p=0.9, do_sample=True)
            response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            responses[question] = response
        
        wandb.log({"model_responses": responses})
 """

' import wandb\nfrom transformers import TrainerCallback\nimport torch\nfrom unsloth.chat_templates import get_chat_template\nfrom unsloth import FastLanguageModel\n\nclass WandBQuestionCallback(TrainerCallback):\n    def __init__(self, tokenizer, model, questions, log_interval=500,**kwargs):\n        self.tokenizer = get_chat_template(\n            tokenizer,\n            chat_template="chatml",\n            mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},\n            map_eos_token=True,\n        )\n        \n        self.model = model\n\n        self.device = "cuda"\n        self.questions = questions  # List of question strings\n        self.log_interval = log_interval\n        \n    def on_step_end(self, args, state, control, **kwargs):\n        # if state.global_step % self.log_interval == 0:\n        #     wandb.log({"step": state.global_step})\n        #     self.log_model_responses()\n        if True:\n            wandb.log({"step": state.glob

In [21]:
""" # Örnek sorular
questions = [
    "433 * b - 7420490 = -7413995 denklemini çöz.",
    "Türkiye'nin başkenti neresidir?",
    "E=mc^2 denkleminin fiziksel anlamı nedir?",
    "Merhaba.Nasılsın?",
    "Merhaba, dün diş çekimi yapıldıktan sonra bu sabah aşırı kanama ile hekime başvurdum. Pihtinin oluştuğunu, ancak kanamanın durmadığı gerekçesiyle dikiş işlemi uyguladı. Bugün herhangi bir kanama veya ağrı yok, yalnız dikiş bölgesinde mukusa benzer bir doku oluştu. Tekrar gitmem gerekir mi?",
    "Merhaba, ben 18 yaşındayım, geçen yıl elimin üst kısmı yanmıştı, şimdi iyileşti ancak elimin üstünde yanık izi kaldı. Bu iz için herhangi bir ilaç veya farklı tedavi yöntemi var mıdır?"
    "Mulan filminin hikayesi hangi kaynaktan esinlenmiştir?",
    "Kartografya günümüzde nasıl teknolojilerden faydalanıyor?"

]

# Callback'i oluştur
wandb_callback = WandBQuestionCallback(tokenizer, model, questions) """

' # Örnek sorular\nquestions = [\n    "433 * b - 7420490 = -7413995 denklemini çöz.",\n    "Türkiye\'nin başkenti neresidir?",\n    "E=mc^2 denkleminin fiziksel anlamı nedir?",\n    "Merhaba.Nasılsın?",\n    "Merhaba, dün diş çekimi yapıldıktan sonra bu sabah aşırı kanama ile hekime başvurdum. Pihtinin oluştuğunu, ancak kanamanın durmadığı gerekçesiyle dikiş işlemi uyguladı. Bugün herhangi bir kanama veya ağrı yok, yalnız dikiş bölgesinde mukusa benzer bir doku oluştu. Tekrar gitmem gerekir mi?",\n    "Merhaba, ben 18 yaşındayım, geçen yıl elimin üst kısmı yanmıştı, şimdi iyileşti ancak elimin üstünde yanık izi kaldı. Bu iz için herhangi bir ilaç veya farklı tedavi yöntemi var mıdır?"\n    "Mulan filminin hikayesi hangi kaynaktan esinlenmiştir?",\n    "Kartografya günümüzde nasıl teknolojilerden faydalanıyor?"\n\n]\n\n# Callback\'i oluştur\nwandb_callback = WandBQuestionCallback(tokenizer, model, questions) '

In [23]:
import wandb

wandb.init(project="Basic LLM Train", id="ecibz7e4", resume="allow")
wandb.watch(model, log="all")


In [24]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    args = TrainingArguments(
       
        gradient_accumulation_steps = 4,
        num_train_epochs=20,  
        per_device_train_batch_size=32,       # GPU başına batch boyutu
        per_device_eval_batch_size=32,       # GPU başına batch boyutu
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 300,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "Unsloth-SmolLM2-1.7B",
        report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
        save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
        save_steps=300,
        warmup_steps=1000,           # İlk 1000 adımda LR'yi yavaş yavaş artır
        run_name="ecibz7e4"
    ),
)

Tokenizing eval dataset (num_proc=4): 100%|██████████| 515101/515101 [01:15<00:00, 6858.60 examples/s] 


In [ ]:
""" from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
) """

' from unsloth.chat_templates import train_on_responses_only\ntrainer = train_on_responses_only(\n    trainer,\n    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",\n    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",\n) '

In [ ]:
""" tokenizer.decode(trainer.train_dataset[5]["input_ids"]) """

' tokenizer.decode(trainer.train_dataset[5]["input_ids"]) '

In [ ]:
""" 
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]) 
"""

' \nspace = tokenizer(" ", add_special_tokens = False).input_ids[0]\ntokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]) \n'

In [ ]:
wandb.watch(model, log="all")
trainer_stats = trainer.train(resume_from_checkpoint="outputs/checkpoint-1200")

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpo

{'loss': 1.3356, 'grad_norm': 0.2179705649614334, 'learning_rate': 0.0001996884152801147, 'epoch': 0.09}


  1%|          | 1800/321940 [7:37:24<4150:34:28, 46.67s/it]

{'loss': 1.2889, 'grad_norm': 0.2604367733001709, 'learning_rate': 0.00019950146444818348, 'epoch': 0.11}


  1%|          | 2100/321940 [11:25:58<4139:33:08, 46.59s/it]

{'loss': 1.2558, 'grad_norm': 0.22182312607765198, 'learning_rate': 0.00019931451361625225, 'epoch': 0.13}


  1%|          | 2136/321940 [11:53:46<3987:43:13, 44.89s/it]

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value":"Fibonacci dizisini devam ettirirmisin : 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<|im_start|>user\nFibonacci dizisini devam ettirirmisin.Sadece cevabı ver lütfen : 1, 1, 2, 3, 5, 8,<|im_end|>\n<|im_start|>assistant\nFibonacci dizisi devam ettirmek için, Fibonacci dizisi için ilk iki değer 1 ve 1 arasında bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak için bir dizi oluşturmak içi